In [5]:
import pandas as pd
import numpy as np
import math
import datetime as dt
from sklearn.metrics import mean_squared_error, mean_absolute_error, explained_variance_score, r2_score 
from sklearn.metrics import mean_poisson_deviance, mean_gamma_deviance, accuracy_score
from sklearn.preprocessing import MinMaxScaler

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM, GRU

from itertools import cycle

# ! pip install plotly
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots


<a name="dataset"></a>
### Import dataset 

In [6]:
# Import dataset
bist100 = pd.read_csv("ASII_stock_combined.csv")
bist100.head()

,timestamp,open,low,high,close,volume
0,2019-01-01,8225,8225,8225,8225,0
1,2019-01-02,8200,8125,8275,8200,14014100
2,2019-01-03,8100,8100,8400,8375,25148400
3,2019-01-04,8400,8200,8450,8350,32475900
4,2019-01-07,8450,8150,8475,8150,32649700


<a name="cname"></a>
### Rename columns

In [7]:
# Rename columns
bist100.rename(columns={"timestamp":"date","Open":"open","High":"high","Low":"low","Close":"close"}, inplace= True)
bist100.head()

,date,open,low,high,close,volume
0,2019-01-01,8225,8225,8225,8225,0
1,2019-01-02,8200,8125,8275,8200,14014100
2,2019-01-03,8100,8100,8400,8375,25148400
3,2019-01-04,8400,8200,8450,8350,32475900
4,2019-01-07,8450,8150,8475,8150,32649700


<a name="dateformat"></a>

### Convert date from string to date format

In [8]:
# convert date field from string to Date format and make it index
bist100['date'] = pd.to_datetime(bist100.date)
bist100.head()

,date,open,low,high,close,volume
0,2019-01-01,8225,8225,8225,8225,0
1,2019-01-02,8200,8125,8275,8200,14014100
2,2019-01-03,8100,8100,8400,8375,25148400
3,2019-01-04,8400,8200,8450,8350,32475900
4,2019-01-07,8450,8150,8475,8150,32649700


In [9]:
bist100.isnull().sum()

date      0
open      0
low       0
high      0
close     0
volume    0
dtype: int64

In [10]:
bist100.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1374 entries, 0 to 1373
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   date    1374 non-null   datetime64[ns]
 1   open    1374 non-null   int64         
 2   low     1374 non-null   int64         
 3   high    1374 non-null   int64         
 4   close   1374 non-null   int64         
 5   volume  1374 non-null   int64         
dtypes: datetime64[ns](1), int64(5)
memory usage: 64.5 KB


<a name="sortdate"></a>

### Sorting dataset by date format

In [11]:
bist100.sort_values(by='date', inplace=True)
bist100.head()

,date,open,low,high,close,volume
0,2019-01-01,8225,8225,8225,8225,0
1,2019-01-02,8200,8125,8275,8200,14014100
2,2019-01-03,8100,8100,8400,8375,25148400
3,2019-01-04,8400,8200,8450,8350,32475900
4,2019-01-07,8450,8150,8475,8150,32649700


In [12]:
bist100.shape

(1374, 6)

<a name="duration"></a>

### Get the duration of dataset

In [13]:
print("Starting date: ",bist100.iloc[0][0])
print("Ending date: ", bist100.iloc[-1][0])
print("Duration: ", bist100.iloc[-1][0]-bist100.iloc[0][0])

Starting date:  2019-01-01 00:00:00
Ending date:  2024-06-14 00:00:00
Duration:  1991 days 00:00:00


C:\Users\pices\AppData\Local\Temp\ipykernel_6068\3850754240.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print("Starting date: ",bist100.iloc[0][0])
C:\Users\pices\AppData\Local\Temp\ipykernel_6068\3850754240.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print("Ending date: ", bist100.iloc[-1][0])
C:\Users\pices\AppData\Local\Temp\ipykernel_6068\3850754240.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print("Durat

<a name="norm"></a>

### Normalizing / scaling close value between 0 to 1

In [14]:
closedf = bist100[['date','close']]
print("Shape of close dataframe:", closedf.shape)

Shape of close dataframe: (1374, 2)


In [15]:
close_stock = closedf.copy()
del closedf['date']
scaler=MinMaxScaler(feature_range=(0,1))
closedf=scaler.fit_transform(np.array(closedf).reshape(-1,1))
print(closedf.shape)

(1374, 1)


<a name="splitdata"></a>

### Split data for training and testing
Ratio for training and testing data is 80:20

In [16]:
training_size=int(len(closedf)*0.80)
test_size=len(closedf)-training_size
train_data,test_data=closedf[0:training_size,:],closedf[training_size:len(closedf),:1]
print("train_data: ", train_data.shape)
print("test_data: ", test_data.shape)

train_data:  (1099, 1)
test_data:  (275, 1)


<a name="tsp"></a>

### Create new dataset according to requirement of time-series prediction 

In [17]:
# convert an array of values into a dataset matrix
def create_dataset(dataset, time_step=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-time_step-1):
        a = dataset[i:(i+time_step), 0]   ###i=0, 0,1,2,3-----99   100 
        dataX.append(a)
        dataY.append(dataset[i + time_step, 0])
    return np.array(dataX), np.array(dataY)

In [18]:
# reshape into X=t,t+1,t+2,t+3 and Y=t+4
time_step = 15
X_train, y_train = create_dataset(train_data, time_step)
X_test, y_test = create_dataset(test_data, time_step)

print("X_train: ", X_train.shape)
print("y_train: ", y_train.shape)
print("X_test: ", X_test.shape)
print("y_test", y_test.shape)

X_train:  (1083, 15)
y_train:  (1083,)
X_test:  (259, 15)
y_test (259,)


<a name="lstm"></a>

### LSTM

In [19]:
# reshape input to be [samples, time steps, features] which is required for LSTM
X_train =X_train.reshape(X_train.shape[0],X_train.shape[1] , 1)
X_test = X_test.reshape(X_test.shape[0],X_test.shape[1] , 1)

print("X_train: ", X_train.shape)
print("X_test: ", X_test.shape)

X_train:  (1083, 15, 1)
X_test:  (259, 15, 1)


<a name="lstmevalmat"></a>

#### LSTM model structure

In [20]:
tf.keras.backend.clear_session()
model=Sequential()
model.add(LSTM(32,return_sequences=True,input_shape=(time_step,1)))
model.add(LSTM(32,return_sequences=True))
model.add(LSTM(32))
model.add(Dense(1))
model.compile(loss='mean_squared_error',optimizer='adam')

In [21]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 15, 32)            4352      
                                                                 
 lstm_1 (LSTM)               (None, 15, 32)            8320      
                                                                 
 lstm_2 (LSTM)               (None, 32)                8320      
                                                                 
 dense (Dense)               (None, 1)                 33        
                                                                 
Total params: 21025 (82.13 KB)
Trainable params: 21025 (82.13 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [22]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=50,batch_size=5,verbose=1)

Epoch 1/50

217/217 [==============================] - 7s 15ms/step - loss: 0.0128 - val_loss: 0.0039
Epoch 2/50
217/217 [==============================] - 2s 9ms/step - loss: 0.0041 - val_loss: 0.0017
Epoch 3/50
217/217 [==============================] - 2s 10ms/step - loss: 0.0036 - val_loss: 0.0013
Epoch 4/50
217/217 [==============================] - 2s 9ms/step - loss: 0.0027 - val_loss: 0.0014
Epoch 5/50
217/217 [==============================] - 2s 10ms/step - loss: 0.0023 - val_loss: 0.0013
Epoch 6/50
217/217 [==============================] - 2s 10ms/step - loss: 0.0019 - val_loss: 0.0011
Epoch 7/50
217/217 [==============================] - 2s 10ms/step - loss: 0.0016 - val_loss: 0.0012
Epoch 8/50
217/217 [==============================] - 2s 10ms/step - loss: 0.0015 - val_loss: 0.0015
Epoch 9/50
217/217 [==============================] - 2s 10ms/step - loss: 0.0013 - val_loss: 8.9794e-04
Epoch 10/50
217/217 [==============================] - 2s 9ms/step - loss: 0.0012 - val_

In [23]:
### Lets Do the prediction and check performance metrics
train_predict=model.predict(X_train)
test_predict=model.predict(X_test)
train_predict.shape, test_predict.shape

9/9 [==============================] - 0s 4ms/step


((1083, 1), (259, 1))

In [24]:
# Transform back to original form

train_predict = scaler.inverse_transform(train_predict)
test_predict = scaler.inverse_transform(test_predict)
original_ytrain = scaler.inverse_transform(y_train.reshape(-1,1)) 
original_ytest = scaler.inverse_transform(y_test.reshape(-1,1)) 

<a name="lstmevalmat"></a>

#### Evaluation metrices RMSE, MSE and MAE

Root Mean Square Error (RMSE), Mean Square Error (MSE) and Mean absolute Error (MAE) are a standard way to measure the error of a model in predicting quantitative data.

In [25]:
# Evaluation metrices RMSE and MAE
print("Train data RMSE: ", math.sqrt(mean_squared_error(original_ytrain,train_predict)))
print("Train data MSE: ", mean_squared_error(original_ytrain,train_predict))
print("Train data MAE: ", mean_absolute_error(original_ytrain,train_predict))
print("Train data R2 score:", r2_score(original_ytrain, train_predict))
print("-------------------------------------------------------------------------------------")
print("Test data RMSE: ", math.sqrt(mean_squared_error(original_ytest,test_predict)))
print("Test data MSE: ", mean_squared_error(original_ytest,test_predict))
print("Test data MAE: ", mean_absolute_error(original_ytest,test_predict))
print("Test data R2 score:", r2_score(original_ytest, test_predict))

Train data RMSE:  121.29444160895903
Train data MSE:  14712.341565229171
Train data MAE:  89.71831739994518
Train data R2 score: 0.9844686828997342
-------------------------------------------------------------------------------------
Test data RMSE:  97.71228950917798
Test data MSE:  9547.691521125413
Test data MAE:  69.54755369208492
Test data R2 score: 0.9808064430915673


<a name="lstmcomparechart"></a>

#### Comparision between original stock close price vs predicted close price

In [26]:
# shift train predictions for plotting

look_back=time_step
trainPredictPlot = np.empty_like(closedf)
trainPredictPlot[:, :] = np.nan
trainPredictPlot[look_back:len(train_predict)+look_back, :] = train_predict
print("Train predicted data: ", trainPredictPlot.shape)

# shift test predictions for plotting
testPredictPlot = np.empty_like(closedf)
testPredictPlot[:, :] = np.nan
testPredictPlot[len(train_predict)+(look_back*2)+1:len(closedf)-1, :] = test_predict
print("Test predicted data: ", testPredictPlot.shape)

names = cycle(['Original close price','Train predicted close price','Test predicted close price'])


plotdf = pd.DataFrame({'date': close_stock['date'],
                       'original_close': close_stock['close'],
                      'train_predicted_close': trainPredictPlot.reshape(1,-1)[0].tolist(),
                      'test_predicted_close': testPredictPlot.reshape(1,-1)[0].tolist()})

fig = px.line(plotdf,x=plotdf['date'], y=[plotdf['original_close'],plotdf['train_predicted_close'],
                                          plotdf['test_predicted_close']],
              labels={'value':'Stock price','date': 'Date'})
fig.update_layout(title_text='Comparision between original close price vs predicted close price',
                  plot_bgcolor='white', font_size=15, font_color='black', legend_title_text='Close Price')
fig.for_each_trace(lambda t:  t.update(name = next(names)))

fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()

Train predicted data:  (1374, 1)
Test predicted data:  (1374, 1)


<a name="lstm_gru"></a>

### GRU

In [27]:
# reshape input to be [samples, time steps, features] which is required for LSTM
X_train =X_train.reshape(X_train.shape[0],X_train.shape[1] , 1)
X_test = X_test.reshape(X_test.shape[0],X_test.shape[1] , 1)

print("X_train: ", X_train.shape)
print("X_test: ", X_test.shape)

X_train:  (1083, 15, 1)
X_test:  (259, 15, 1)


In [28]:
model=Sequential()
model.add(GRU(32,return_sequences=True,input_shape=(time_step,1)))
model.add(GRU(32,return_sequences=True))
model.add(GRU(32))
model.add(Dense(1))
model.compile(loss='mean_squared_error',optimizer='adam')

In [29]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru (GRU)                   (None, 15, 32)            3360      
                                                                 
 gru_1 (GRU)                 (None, 15, 32)            6336      
                                                                 
 gru_2 (GRU)                 (None, 32)                6336      
                                                                 
 dense_1 (Dense)             (None, 1)                 33        
                                                                 
Total params: 16065 (62.75 KB)
Trainable params: 16065 (62.75 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [30]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=50,batch_size=5,verbose=1)

Epoch 1/50
217/217 [==============================] - 9s 18ms/step - loss: 0.0108 - val_loss: 0.0010
Epoch 2/50
217/217 [==============================] - 3s 13ms/step - loss: 0.0012 - val_loss: 6.8167e-04
Epoch 3/50
217/217 [==============================] - 3s 13ms/step - loss: 0.0010 - val_loss: 5.0815e-04
Epoch 4/50
217/217 [==============================] - 3s 13ms/step - loss: 8.7717e-04 - val_loss: 4.8765e-04
Epoch 5/50
217/217 [==============================] - 3s 13ms/step - loss: 8.5447e-04 - val_loss: 3.8977e-04
Epoch 6/50
217/217 [==============================] - 3s 13ms/step - loss: 6.9318e-04 - val_loss: 3.9703e-04
Epoch 7/50
217/217 [==============================] - 3s 13ms/step - loss: 7.3191e-04 - val_loss: 3.9481e-04
Epoch 8/50
217/217 [==============================] - 3s 12ms/step - loss: 6.9194e-04 - val_loss: 4.8935e-04
Epoch 9/50
217/217 [==============================] - 3s 12ms/step - loss: 7.3405e-04 - val_loss: 3.4374e-04
Epoch 10/50
217/217 [==============

In [31]:
### Lets Do the prediction and check performance metrics
train_predict=model.predict(X_train)
test_predict=model.predict(X_test)
train_predict.shape, test_predict.shape

9/9 [==============================] - 0s 4ms/step


((1083, 1), (259, 1))

In [32]:
# Transform back to original form

train_predict = scaler.inverse_transform(train_predict)
test_predict = scaler.inverse_transform(test_predict)
original_ytrain = scaler.inverse_transform(y_train.reshape(-1,1)) 
original_ytest = scaler.inverse_transform(y_test.reshape(-1,1)) 

In [33]:
# Evaluation metrices RMSE and MAE
print("Train data RMSE: ", math.sqrt(mean_squared_error(original_ytrain,train_predict)))
print("Train data MSE: ", mean_squared_error(original_ytrain,train_predict))
print("Train data MAE: ", mean_absolute_error(original_ytrain,train_predict))
print("Train data R2 score:", r2_score(original_ytrain, train_predict))
print("-------------------------------------------------------------------------------------")
print("Test data RMSE: ", math.sqrt(mean_squared_error(original_ytest,test_predict)))
print("Test data MSE: ", mean_squared_error(original_ytest,test_predict))
print("Test data MAE: ", mean_absolute_error(original_ytest,test_predict))
print("Test data R2 score:", r2_score(original_ytest, test_predict))

Train data RMSE:  143.72381325749788
Train data MSE:  20656.534497276127
Train data MAE:  111.0575026690905
Train data R2 score: 0.9781936011988737
-------------------------------------------------------------------------------------
Test data RMSE:  111.33645296086706
Test data MSE:  12395.805757907363
Test data MAE:  83.94593086389962
Test data R2 score: 0.9750809289644677


In [34]:
# shift train predictions for plotting

look_back=time_step
trainPredictPlot = np.empty_like(closedf)
trainPredictPlot[:, :] = np.nan
trainPredictPlot[look_back:len(train_predict)+look_back, :] = train_predict
print("Train predicted data: ", trainPredictPlot.shape)

# shift test predictions for plotting
testPredictPlot = np.empty_like(closedf)
testPredictPlot[:, :] = np.nan
testPredictPlot[len(train_predict)+(look_back*2)+1:len(closedf)-1, :] = test_predict
print("Test predicted data: ", testPredictPlot.shape)

names = cycle(['Original close price','Train predicted close price','Test predicted close price'])


plotdf = pd.DataFrame({'date': close_stock['date'],
                       'original_close': close_stock['close'],
                      'train_predicted_close': trainPredictPlot.reshape(1,-1)[0].tolist(),
                      'test_predicted_close': testPredictPlot.reshape(1,-1)[0].tolist()})

fig = px.line(plotdf,x=plotdf['date'], y=[plotdf['original_close'],plotdf['train_predicted_close'],
                                          plotdf['test_predicted_close']],
              labels={'value':'Stock price','date': 'Date'})
fig.update_layout(title_text='Comparision between original close price vs predicted close price',
                  plot_bgcolor='white', font_size=15, font_color='black', legend_title_text='Close Price')
fig.for_each_trace(lambda t:  t.update(name = next(names)))

fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()

Train predicted data:  (1374, 1)
Test predicted data:  (1374, 1)


<a name="lstm_gru"></a>

### LSTM + GRU

In [35]:
# reshape input to be [samples, time steps, features] which is required for LSTM
X_train =X_train.reshape(X_train.shape[0],X_train.shape[1] , 1)
X_test = X_test.reshape(X_test.shape[0],X_test.shape[1] , 1)

print("X_train: ", X_train.shape)
print("X_test: ", X_test.shape)

X_train:  (1083, 15, 1)
X_test:  (259, 15, 1)


#### Model structure

In [36]:
tf.keras.backend.clear_session()
model=Sequential()
model.add(LSTM(32,return_sequences=True,input_shape=(time_step,1)))
model.add(LSTM(32,return_sequences=True))
model.add(GRU(32,return_sequences=True))
model.add(GRU(32))
model.add(Dense(1))
model.compile(loss='mean_squared_error',optimizer='adam')

In [37]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 15, 32)            4352      
                                                                 
 lstm_1 (LSTM)               (None, 15, 32)            8320      
                                                                 
 gru (GRU)                   (None, 15, 32)            6336      
                                                                 
 gru_1 (GRU)                 (None, 32)                6336      
                                                                 
 dense (Dense)               (None, 1)                 33        
                                                                 
Total params: 25377 (99.13 KB)
Trainable params: 25377 (99.13 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [38]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=50,batch_size=5,verbose=1)

Epoch 1/50
217/217 [==============================] - 10s 22ms/step - loss: 0.0145 - val_loss: 0.0018
Epoch 2/50
217/217 [==============================] - 3s 15ms/step - loss: 0.0041 - val_loss: 0.0024
Epoch 3/50
217/217 [==============================] - 3s 15ms/step - loss: 0.0031 - val_loss: 0.0018
Epoch 4/50
217/217 [==============================] - 3s 15ms/step - loss: 0.0029 - val_loss: 0.0014
Epoch 5/50
217/217 [==============================] - 3s 15ms/step - loss: 0.0021 - val_loss: 0.0010
Epoch 6/50
217/217 [==============================] - 3s 15ms/step - loss: 0.0018 - val_loss: 0.0018
Epoch 7/50
217/217 [==============================] - 3s 15ms/step - loss: 0.0017 - val_loss: 9.1472e-04
Epoch 8/50
217/217 [==============================] - 3s 15ms/step - loss: 0.0013 - val_loss: 0.0012
Epoch 9/50
217/217 [==============================] - 3s 15ms/step - loss: 0.0011 - val_loss: 5.5255e-04
Epoch 10/50
217/217 [==============================] - 3s 16ms/step - loss: 0.0011

In [39]:
### Lets Do the prediction and check performance metrics
train_predict=model.predict(X_train)
test_predict=model.predict(X_test)
train_predict.shape, test_predict.shape

9/9 [==============================] - 0s 5ms/step


((1083, 1), (259, 1))

In [40]:
# Transform back to original form

train_predict = scaler.inverse_transform(train_predict)
test_predict = scaler.inverse_transform(test_predict)
original_ytrain = scaler.inverse_transform(y_train.reshape(-1,1)) 
original_ytest = scaler.inverse_transform(y_test.reshape(-1,1)) 

In [41]:
# Evaluation metrices RMSE and MAE
print("Train data RMSE: ", math.sqrt(mean_squared_error(original_ytrain,train_predict)))
print("Train data MSE: ", mean_squared_error(original_ytrain,train_predict))
print("Train data MAE: ", mean_absolute_error(original_ytrain,train_predict))
print("Train data R2 score:", r2_score(original_ytrain, train_predict))
print("-------------------------------------------------------------------------------------")
print("Test data RMSE: ", math.sqrt(mean_squared_error(original_ytest,test_predict)))
print("Test data MSE: ", mean_squared_error(original_ytest,test_predict))
print("Test data MAE: ", mean_absolute_error(original_ytest,test_predict))
print("Test data R2 score:", r2_score(original_ytest, test_predict))

Train data RMSE:  119.15046365568205
Train data MSE:  14196.83298936401
Train data MAE:  88.35092498412973
Train data R2 score: 0.9850128877174493
-------------------------------------------------------------------------------------
Test data RMSE:  95.98461747031621
Test data MSE:  9213.046790922932
Test data MAE:  68.14829987632723
Test data R2 score: 0.9814791735268811


<a name="lstm_grucomparechart"></a>

#### Comparision between original stock close price vs predicted close price

In [42]:
# shift train predictions for plotting

look_back=time_step
trainPredictPlot = np.empty_like(closedf)
trainPredictPlot[:, :] = np.nan
trainPredictPlot[look_back:len(train_predict)+look_back, :] = train_predict
print("Train predicted data: ", trainPredictPlot.shape)

# shift test predictions for plotting
testPredictPlot = np.empty_like(closedf)
testPredictPlot[:, :] = np.nan
testPredictPlot[len(train_predict)+(look_back*2)+1:len(closedf)-1, :] = test_predict
print("Test predicted data: ", testPredictPlot.shape)

names = cycle(['Original close price','Train predicted close price','Test predicted close price'])

plotdf = pd.DataFrame({'date': close_stock['date'],
                       'original_close': close_stock['close'],
                      'train_predicted_close': trainPredictPlot.reshape(1,-1)[0].tolist(),
                      'test_predicted_close': testPredictPlot.reshape(1,-1)[0].tolist()})

fig = px.line(plotdf,x=plotdf['date'], y=[plotdf['original_close'],plotdf['train_predicted_close'],
                                          plotdf['test_predicted_close']],
              labels={'value':'Stock price','date': 'Date'})
fig.update_layout(title_text='Comparision between original close price vs predicted close price',
                  plot_bgcolor='white', font_size=15, font_color='black', legend_title_text='Close Price')
fig.for_each_trace(lambda t:  t.update(name = next(names)))

fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()

Train predicted data:  (1374, 1)
Test predicted data:  (1374, 1)
